In [1]:
import os
# make sure pyspark tells workers to use python2 not 3 if both are
#installed

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python2'

import pyspark
from pyspark.sql import SQLContext

sc =  pyspark.SparkContext('local[*]')
sqlCtx = SQLContext(sc)

In [2]:
authors = sc.textFile("data/2000s_authors.csv")

In [3]:
import itertools   
coauthors = authors.flatMap(lambda a: [[coauth, 1] for coauth in itertools.combinations(a, 2)])

In [4]:
coauthors.collect()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/opt/conda/envs/python2/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/opt/conda/envs/python2/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server